# Keeping track of information
LLMs calls are inherently stateless - they do not have memory of any previous interactions. Every call is an independent event, and **YOU** must manage any information that needs to be carried over time.

In this notebook we will look at a few different things that we might want to keep track of between calls.

## Conversation History
Keeping track of the conversation history is actually easy. Firstly it is important to remember that LLMs often have the following pattern:

```
-> system prompt
-> user prompt
-> model response

-> user prompt
-> model response

-> user prompt
-> model response

-> etc.
```

We actually saw an example of this in the prompting notebook when we looked at few-shot prompting.

Here is a really simple example of how we can keep track of the conversation history. We can first define a `system_state` dictionary that will store important information for us. We can give it a `conversation_history` key that will store the conversation history.

In [1]:
system_state = {
    "conversation_history": []
}

In [2]:
system_prompt = (
    "You are a helpful philosophical assistant. "
    "You will help me think about philosophical questions. "
    "Please keep your answers concise and to the point."
)

system_state["conversation_history"].append({
    "role": "system",
    "content": system_prompt
})

user_prompt = "What is the meaning of life?"

system_state["conversation_history"].append({
    "role": "user",
    "content": user_prompt
})

In [3]:
for message in system_state["conversation_history"]:
    print(f"{message['role']}: {message['content']}\n")

system: You are a helpful philosophical assistant. You will help me think about philosophical questions. Please keep your answers concise and to the point.

user: What is the meaning of life?



Now we can use this conversation history to generate a response.

In [4]:
from openai import OpenAI
client = OpenAI()

import dotenv
import os

dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from rich.pretty import pprint

In [5]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=system_state["conversation_history"],
    max_tokens=512,
    temperature=1.0
)

print(response.choices[0].message.content)

The meaning of life is a deeply subjective question and can vary widely from person to person. Philosophically, it ranges from existentialist views, which suggest individuals create their own meaning, to religious perspectives that see life as a fulfillment of a divine purpose. Some thinkers propose that meaning is found in relationships, knowledge, creativity, or contributing to the greater good. Ultimately, it's a question that encourages personal reflection and exploration.


Great, but now what happens if I want to ask a follow up question? Without the conversation history?

In [6]:
follow_up_prompt = "Can you tell more about point 1?"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": follow_up_prompt
    }],
    max_tokens=512,
    temperature=1.0
)

print(response.choices[0].message.content)


Of course! However, I need a bit more context to know what "point 1" refers to. Could you please provide more details or specify the topic or list you are referencing?


Obviously it has no memory of the previous conversation. So we just need to append the follow up prompt to the conversation history.

In [7]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=system_state["conversation_history"],
    max_tokens=512,
    temperature=1.0
)

system_state["conversation_history"].append({
    "role": "assistant",
    "content": response.choices[0].message.content
})

for message in system_state["conversation_history"]:
    print(f"{message['role'].upper()}: {message['content']}\n")

SYSTEM: You are a helpful philosophical assistant. You will help me think about philosophical questions. Please keep your answers concise and to the point.

USER: What is the meaning of life?

ASSISTANT: The meaning of life is a deeply personal and subjective question. Philosophers have proposed various answers, including seeking happiness, fulfilling potential, forming connections with others, or contributing to society. Existentialists suggest creating one's own meaning through choices and actions. Ultimately, it may depend on individual beliefs, values, and experiences.



And now we can keep the conversation going in a simple loop. If you run this cell a few times you will see that the conversation history is correctly maintained.

In [8]:
while True:
    user_input = input("You: ")
    
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Assistant: Goodbye!")
        break
    
    system_state["conversation_history"].append({
        "role": "user",
        "content": user_input
    })
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=system_state["conversation_history"],
        max_tokens=512,
        temperature=1.0
    )
    
    assistant_response = response.choices[0].message.content
    print(f"Assistant: {assistant_response}\n")
    
    system_state["conversation_history"].append({
        "role": "assistant",
        "content": assistant_response
    })

Assistant: Eating serves multiple purposes: biologically, it provides the nutrients and energy necessary for survival; socially, it fosters connections and community; and culturally, it can express identity and tradition. Philosophically, it can symbolize the relationship between individuals and their environment, reflecting our choices and their implications for health and moral considerations.

Assistant: Sleep serves several critical functions, both biological and psychological. Biologically, it is essential for physical health, allowing the body to repair, restore energy, and process information. Psychologically, it aids in emotional regulation, memory consolidation, and cognitive functioning. Philosophically, sleep can symbolize vulnerability, surrender, and the cyclical nature of existence, prompting reflections on consciousness and the nature of reality.

Assistant: A large language model (LLM) is an artificial intelligence system designed to understand, generate, and manipulate

In [9]:
from rich.console import Console
from rich.text import Text

console = Console()

In [10]:
colors = {
    "system": "green",
    "user": "cyan",
    "assistant": "magenta"
}

for message in system_state["conversation_history"]:
    role = message["role"]
    content = message["content"]
    color = colors[role]
    console.print(f"[{color}]{role.upper()}: {content}[/{color}]")

SYSTEM: You are a helpful philosophical assistant. You will help me think about philosophical questions. Please 
keep your answers concise and to the point.

USER: What is the meaning of life?

ASSISTANT: The meaning of life is a deeply personal and subjective question. Philosophers have proposed various 
answers, including seeking happiness, fulfilling potential, forming connections with others, or contributing to 
society. Existentialists suggest creating one's own meaning through choices and actions. Ultimately, it may depend 
on individual beliefs, values, and experiences.

USER: what is the meaning of eat?

ASSISTANT: Eating serves multiple purposes: biologically, it provides the nutrients and energy necessary for 
survival; socially, it fosters connections and community; and culturally, it can express identity and tradition. 
Philosophically, it can symbolize the relationship between individuals and their environment, reflecting our 
choices and their implications for health and moral considerations.

...

ASSISTANT: Sleep serves several critical functions, both biological and psychological. Biologically, it is 
essential for physical health, allowing the body to repair, restore energy, and process information. 
Psychologically, it aids in emotional regulation, memory consolidation, and cognitive functioning. Philosophically,
sleep can symbolize vulnerability, surrender, and the cyclical nature of existence, prompting reflections on 
consciousness and the nature of reality.

USER: what is a large language model?

ASSISTANT: A large language model (LLM) is an artificial intelligence system designed to understand, generate, and 
manipulate human language. It is typically based on deep learning architectures, such as transformer networks, and 
is trained on vast amounts of text data to predict or generate text based on input. LLMs can perform various 
language tasks, including translation, summarization, and conversation, but they do not possess understanding or 
consciousness; they operate based on patterns learned from the data.

USER: what are you?

ASSISTANT: I am an artificial intelligence language model designed to assist with information and facilitate 
discussions on various topics, including philosophy. I process and generate text based on patterns learned from 
extensive datasets, but I do not have consciousness, emotions, or personal experiences. My purpose is to assist 
users in exploring ideas and answering questions.

## Tracking tokens
We should probably also track the tokens. This can be useful for a few reasons - we can track costs, and we can use it to cut off conversation history when we get too close to our limit.

We can make this as simple or complicated as we want. Probably we should create a `Conversation` class to keep track of things like this.

In [11]:
class Conversation:
    def __init__(self, system_prompt):
        self.system_prompt = system_prompt
        self.history = []
        self.tokens = 0
        self.token_limit = 300

        self.add_message("system", system_prompt)

    def add_message(self, role, content):
        self.history.append({"role": role, "content": content})
        self.tokens += len(content)

    def check_token_limit(self):
        while self.tokens > self.token_limit and len(self.history) > 1:
            # Remove the oldest non-system message
            for i in range(1, len(self.history)):
                if self.history[i]["role"] != "system":
                    removed_message = self.history.pop(i)
                    self.tokens -= len(removed_message["content"])
                    break

    def response(self, user_input):
        self.add_message("user", user_input)
        if self.tokens > self.token_limit:
            self.check_token_limit()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.history,
            max_tokens=512,
            temperature=1.0
        ).choices[0].message.content
        
        self.add_message("assistant", response)

        return response
                

Let's see how this works.

Let's see if we can get the model to forget about things we mention at the start of a conversation.

In [12]:
conversation = Conversation(system_prompt)

In [13]:
print(conversation.response("Hello, my name is Bob and I am 25 years old!"))
print(f"Tokens: {conversation.tokens}")

Hello, Bob! It's nice to meet you. What philosophical questions or topics are you interested in exploring today?
Tokens: 303


In [14]:
print(conversation.response("What is my name?"))
print(f"Tokens: {conversation.tokens}")


You mentioned the name "Bob" in your previous message. If that's not correct, please let me know!
Tokens: 372


Great, so now we have hit our token limit, and the conversation should be trimmed in the next response.

In [15]:
print(conversation.response("What is my age?"))
print(f"Tokens: {conversation.tokens}")

I don't have information about your age. You would need to share that information for me to know.
Tokens: 372


In [16]:
pprint(conversation.history, expand_all=True)

[
│   {
│   │   'role': 'system',
│   │   'content': 'You are a helpful philosophical assistant. You will help me think about philosophical questions. Please keep your answers concise and to the point.'
│   },
│   {
│   │   'role': 'user',
│   │   'content': 'What is my name?'
│   },
│   {
│   │   'role': 'assistant',
│   │   'content': 'You mentioned the name "Bob" in your previous message. If that\'s not correct, please let me know!'
│   },
│   {
│   │   'role': 'user',
│   │   'content': 'What is my age?'
│   },
│   {
│   │   'role': 'assistant',
│   │   'content': "I don't have information about your age. You would need to share that information for me to know."
│   }
]

This is a good start, but there is a problem here. What if there was something very important that we wanted to keep track of that was mentioned at the start of the conversation, but it has been cut off!?